# Project Group - 4

## Members and student numbers:
Hsuan-An Chu: 5914647\
Jarrik Overbosch: 6105734\
Mats Poppe: 5883245\
Cian Rippen: 5054141\
Sam Terstappen: 6078720


 

# Research Objective
## What is the influence of the weather on travel behaviour in The Netherlands?
* What is the correlation between amount of percipitation and amount of transport passengers, based on OV chipcard checkins? 
* What is the effect of weather on road traffic?  
* What is the correlation between the weather and congestion?
* Is there a difference in behaviour between cities and rural areas?
* Where do the tipping points of behaviour lie? (e.g.amount of rain that starts people to change their behaviour)




*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

OV Chipcard checkins \
https://www.translink.nl/library \
Weather data \
https://dataplatform.knmi.nl/group/precipitation \
https://www.daggegevens.knmi.nl/klimatologie/uurgegevens \
Congestion data \
https://downloads.rijkswaterstaatdata.nl/filedata/


# Data Pipeline

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Congestion data

The following code was used to convert 10 Excel files containing congestion data of each month of 2023 up until October to one Pickle file.
It was run once, and then commented as it is not necessary to run it again.

In [52]:
# df_cong = pd.DataFrame()
# months = ['jan', 'feb', 'mar', 'apr', 'mei', 'jun', 'jul', 'aug', 'sep', 'okt']
# for i in months:",
#     month = pd.read_excel("Literature and Data/Congestion_data_2022/" + i + ".xlsx)
#     df_cong = pd.concat([df_cong, month])
#     print(i)

# df_cong.to_pickle('Literature and data/df_cong_pickle.pkl')

In the following code the pickle file is read and assigned to a DataFrame. Several relevant columns are extracted from the numerous columns in the original DF. Subsequently the commas designating decimal points in the 'FileZwaarte' column are replaced with dots, so that they can be summed to obtain the total 'FileZwaarte' per day. FileZwaarte is a variable that expresses the severity of a traffic jam in kilometres * minutes, so it accounts for both the length and duration of a traffic jam. This is in out opinion a good indicator for the severity of a traffic jam, as opposed to just the length or duration of it. 

In [2]:
unpickled_df_cong = pd.read_pickle('Literature and data/df_cong_pickle.pkl').reset_index() 
df_cong_filt = unpickled_df_cong[['DatumFileBegin', 'TijdFileBegin', 'TijdFileEind', 'FileZwaarte', 'Oorzaak_4']]
df_cong_filt['FileZwaarte'] = df_cong_filt['FileZwaarte'].str.replace(',', '.', regex=True).astype(float)
# df_cong_filt.head(30)
df_cong_grouped = df_cong_filt.groupby('DatumFileBegin')['FileZwaarte'].sum().reset_index()
df_cong_grouped.head()
#The warning this gives is not a thing to worry about. 

/var/folders/j9/9tvcq7qs4gvf1y4c24tlh03h0000gn/T/ipykernel_60262/2544055318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cong_filt['FileZwaarte'] = df_cong_filt['FileZwaarte'].str.replace(',', '.', regex=True).astype(float)


,DatumFileBegin,FileZwaarte
0,2023-01-01,93.500
1,2023-01-02,2549.248
2,2023-01-03,4404.093
3,2023-01-04,6634.145
4,2023-01-05,7113.570


The congestion severity data is displayed in a graph.Two data points with severe bad weather are highlighted. It can be seen that these days are outliers in terms of congestion. It should be noted that on April 6th, the congestion was also made worse because a lot of people were travelling due to the Easter weekend. 

In [3]:
fig = px.line(df_cong_grouped, x="DatumFileBegin", y="FileZwaarte", title='Congestion severity per day')
fig.update_layout(xaxis_title="Date", yaxis_title="Congestion severity (km*min)")
# df_cong_grouped['DatumFileBegin'] = pd.to_datetime(df_cong_grouped['DatumFileBegin'])
# print(type(df_cong_grouped.iloc[95,0]))
fig.add_trace(
    go.Scatter(
        x=[df_cong_grouped.iloc[95,0], df_cong_grouped.iloc[18,0]],
        y=[df_cong_grouped.iloc[95,1], df_cong_grouped.iloc[18,1]],
        mode="markers",
        name="Extraordinarily bad weather days",
        showlegend=True)
)
fig.update_layout(legend=dict(
    orientation="h", 
    xanchor="right",
    y=1,
    x=1
))
fig.show()

### OV Chipkaart Data

In the following code, the Excel file containing OV (public transport) check-in data is read, and the data is modified. The number of check ins is multiplied by 1000, as the data was supplied per 1000 check ins. Also, the number of checkins was summed per day. A graph displaying check in data per day is shown. 

In [4]:
#reading
file = "Literature and Data/20230908_Instappers_per_uur_export_V3.csv"
df_OV = pd.read_csv(file)

#processing
df_OV["Aantal_check_ins"] = df_OV["Aantal_check_ins"] * 1000
df_OV["Aantal_check_ins"] = df_OV["Aantal_check_ins"].astype('int')
df_OV_sum = df_OV.groupby(by="Datum", sort=False)["Aantal_check_ins"].sum().reset_index()
df_OV_sum["Datum"] = pd.to_datetime(df_OV_sum["Datum"], format="%d-%m-%Y")
df_OV_sum.head()

#plotting
fig = px.line(df_OV_sum, x="Datum", y="Aantal_check_ins", title='Number of check-ins in 2023')
fig.update_layout(xaxis_title="Date", yaxis_title="Number of OV check-ins")
fig.show()

### Weather Data

In the following code, the CSV file containing weather data from 50 KNMI weather stations is used, some of the weather stations doesn't have rain recording equipments. The location and data recorded by the stations are plot as the following.  and the data is group and averages by dates. Then adjustment were made beacuase some of the data were in 0.1 increments Graphs displaying all types of weather data are shown .

In [5]:
#reading data
df_weather = pd.read_csv("Literature and data/KNMI_daily_0123-0923.csv")
df_station_name = pd.read_csv("Literature and data/KNMI_station.csv")
df_weather.head()

,# STN,YYYYMMDD,FG,FXX,TG,DR,RH,VVN
0,209,20230101,96,200,,,,
1,209,20230102,69,100,,,,
2,209,20230103,88,200,,,,
3,209,20230104,141,220,,,,
4,209,20230105,90,160,,,,


In [6]:
#plotting the weather stations(do not rerun this cell, mapbox token need to be secured)
df_weather.rename(columns={"# STN":"STN"}, inplace=True)
df_station_name.rename(columns={"STN         ":"STN"}, inplace=True)
df_weather["STN"].astype(int)
df_station_name["STN"].astype(int)

df_station_data = df_station_name.merge(df_weather, on="STN")
df_station_data.drop_duplicates(subset="STN",inplace=True)
df_station_data["   RH"] = pd.to_numeric(df_station_data["   RH"], errors="coerce")
df_station_data["hasraindata"] = np.where(df_station_data["   RH"]>=0, True, False)
df_station_data = df_station_data.iloc[:,[0,1,2,3,4,12]]
df_station_data.drop_duplicates(inplace=True)

px.set_mapbox_access_token("pk.eyJ1IjoiaHN1YW4tc2hhbmUiLCJhIjoiY2xvMnB3b2NqMDl3YzJpbW56eWxnNHRrNSJ9.zJi-0PqhXOPzbqZ973FdxA")
fig = px.scatter_mapbox(df_station_data, lat="LAT(north)  ", lon="LON(east)   ", color="hasraindata", zoom = 5, hover_name="NAME", labels={"hasraindata":"Rain data collection"})
fig.update_layout(title = "Weather Station Data used")
fig.show()

In [7]:
#precess data
df_weather.columns = ["stations", "dates", "windspeed","windspeed_max","temperature","rain_duration","rain_amount", "visibility"]#rename col
df_weather = df_weather.drop(columns=["stations"])
df_weather[df_weather.columns.difference(["dates"])] = df_weather[df_weather.columns.difference(["dates"])].apply(pd.to_numeric, errors='coerce')
df_weather.head()

df_weather_mean = df_weather.groupby(["dates"], as_index=False).mean(numeric_only=True)
df_weather_mean[df_weather_mean.columns.difference(["dates", "visibility"])] = df_weather_mean[df_weather_mean.columns.difference(["dates", "visibility"])]*0.1


df_weather_mean["dates"] = pd.to_datetime(df_weather_mean["dates"], format="%Y%m%d")
df_weather_mean.head()

,dates,windspeed,windspeed_max,temperature,rain_duration,rain_amount,visibility
0,2023-01-01,7.282609,18.608696,11.714706,5.363636,4.330303,52.36
1,2023-01-02,4.986957,12.369565,8.447059,4.651515,3.918182,38.52
2,2023-01-03,6.228261,16.152174,6.338235,1.469697,1.424242,14.36
3,2023-01-04,10.956522,19.521739,11.044118,8.351515,11.018182,33.64
4,2023-01-05,6.521739,14.217391,10.144118,1.621212,0.818182,36.24


In [8]:
#plot the weather
fig = px.line(df_weather_mean, x="dates", y="windspeed")
fig.show()

In [9]:
fig = px.line(df_weather_mean, x="dates", y="temperature")
fig.show()

In [10]:
fig = px.line(df_weather_mean, x="dates", y="rain_amount")
fig.show()

# Weather and Congestion correlation 

To plot the weather and congestiondata together we first need to merg the two dataframes

In [11]:
df_cong_grouped.rename(columns={"DatumFileBegin":"dates"}, inplace=True)
df_cong_weather = df_cong_grouped.merge(df_weather_mean, on = "dates")
df_cong_weather.head()

,dates,FileZwaarte,windspeed,windspeed_max,temperature,rain_duration,rain_amount,visibility
0,2023-01-01,93.500,7.282609,18.608696,11.714706,5.363636,4.330303,52.36
1,2023-01-02,2549.248,4.986957,12.369565,8.447059,4.651515,3.918182,38.52
2,2023-01-03,4404.093,6.228261,16.152174,6.338235,1.469697,1.424242,14.36
3,2023-01-04,6634.145,10.956522,19.521739,11.044118,8.351515,11.018182,33.64
4,2023-01-05,7113.570,6.521739,14.217391,10.144118,1.621212,0.818182,36.24


In [12]:
#congestion v rain
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_cong_weather["dates"], y=df_cong_weather["FileZwaarte"], name="Congestions"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_cong_weather["dates"], y=df_cong_weather["rain_amount"], name="Rain amount"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Rain amount vs Congestion severity"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Congestion severity (km*min)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Avg Rain amount(mm/h)</b>", secondary_y=True)

fig.show()

In [13]:
#congestion v rain
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_cong_weather["dates"], y=df_cong_weather["FileZwaarte"], name="Congestions"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_cong_weather["dates"], y=df_cong_weather["windspeed"], name="Wind speed"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Wind speed vs Congestion severity"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Congestion severity (km*min)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Avg Wind Speed(m/s)</b>", secondary_y=True)

fig.show()

In [14]:
#Congestion v Temps
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_cong_weather["dates"], y=df_cong_weather["FileZwaarte"], name="Congestions"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_cong_weather["dates"], y=df_cong_weather["temperature"], name="Temperature"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Temperature vs Congestion severity"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Congestion severity (km*min)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Avg Temperature(C)</b>", secondary_y=True)

fig.show()

In [29]:
#correlation test all
cong_corr_rains = df_cong_weather["rain_amount"].corr(df_cong_weather["FileZwaarte"])
cong_corr_wind = df_cong_weather["windspeed"].corr(df_cong_weather["FileZwaarte"])
cong_corr_temps = df_cong_weather["temperature"].corr(df_cong_weather["FileZwaarte"])

cong_corr_data = {"WeatherTypes": ["Rain", "Wind", "Temps"], "Correlation": [cong_corr_rains,cong_corr_wind,cong_corr_temps]}
df_cong_corr = pd.DataFrame(cong_corr_data)

fig = px.bar(df_cong_corr, x="WeatherTypes", y="Correlation", color="WeatherTypes",text_auto=True, title="Correlation between Weather condiditions and congestion(All data)")
fig.show()

Investigate further in the weekends

In [25]:
df_cong_grouped['dates'] = pd.to_datetime(df_cong_grouped['dates'])
df_cong_grouped['DayOfWeek'] = df_cong_grouped['dates'].dt.day_name()

# Filter gegevens voor weekenddagen in 2023
df_cong_weekends = df_cong_grouped[(df_cong_grouped['DayOfWeek'].isin(['Saturday', 'Sunday']))]

fig = px.line(df_cong_weekends, x="dates", y="FileZwaarte", title='Congestion in the weekends')
fig.update_layout(xaxis_title="Date", yaxis_title="Congestion severity (km*min)")

df_cong_weekends.head()
fig.show()

In [37]:
df_cong_weekends.rename(columns={"DatumFileBegin":"dates"}, inplace=True)
df_cong_w_weekends = df_cong_weekends.merge(df_weather_mean, on = "dates")

#correlation test weekends
cong_corr_rains_weekends = df_cong_w_weekends["rain_amount"].corr(df_cong_w_weekends["FileZwaarte"])
cong_corr_wind_weekends = df_cong_w_weekends["windspeed"].corr(df_cong_w_weekends["FileZwaarte"])
cong_corr_temps_weekends = df_cong_w_weekends["temperature"].corr(df_cong_w_weekends["FileZwaarte"])

cong_corr_data_weekends = {"WeatherTypes": ["Rain", "Wind", "Temps"], "Correlation": [cong_corr_rains_weekends,cong_corr_wind_weekends,cong_corr_temps_weekends]}
df_cong_corr_weekends = pd.DataFrame(cong_corr_data_weekends)

fig = px.bar(df_cong_corr_weekends, x="WeatherTypes", y="Correlation", color="WeatherTypes",text_auto=True, title="Correlation between Weather condiditions and congestion(weekends data)")
fig.show()

/var/folders/j9/9tvcq7qs4gvf1y4c24tlh03h0000gn/T/ipykernel_60262/4272539318.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Weather and OV check-ins correlation 

To plot the weather and ov data together we first need to merg the two dataframes

In [31]:
df_OV_sum.rename(columns={"Datum":"dates"}, inplace=True)
df_OV_weather = df_OV_sum.merge(df_weather_mean, on = "dates")
df_OV_weather.head()

,dates,Aantal_check_ins,windspeed,windspeed_max,temperature,rain_duration,rain_amount,visibility
0,2023-01-01,1003699,7.282609,18.608696,11.714706,5.363636,4.330303,52.36
1,2023-01-02,2074400,4.986957,12.369565,8.447059,4.651515,3.918182,38.52
2,2023-01-03,2465899,6.228261,16.152174,6.338235,1.469697,1.424242,14.36
3,2023-01-04,2446900,10.956522,19.521739,11.044118,8.351515,11.018182,33.64
4,2023-01-05,2643299,6.521739,14.217391,10.144118,1.621212,0.818182,36.24


In [32]:
#OV v Rain
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_OV_weather["dates"], y=df_OV_weather["Aantal_check_ins"], name="Check-ins"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_OV_weather["dates"], y=df_OV_weather["rain_amount"], name="Rain amount"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Rain Amount vs Check-ins"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Check-ins</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Avg Rain amount(mm/h)</b>", secondary_y=True)

fig.show()

In [68]:
#OV v wind
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_OV_weather["dates"], y=df_OV_weather["Aantal_check_ins"], name="Check-ins"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_OV_weather["dates"], y=df_OV_weather["windspeed"], name="Wind Speed"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Wind speed vs Check-ins"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Check-ins</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Avg Wind Speed(m/s)</b>", secondary_y=True)

fig.show()

In [69]:
#OV v Temps
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_OV_weather["dates"], y=df_OV_weather["Aantal_check_ins"], name="Check-ins"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_OV_weather["dates"], y=df_OV_weather["temperature"], name="Temperature"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Temperature vs Check-ins"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Check-ins</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Avg temperature(C)</b>", secondary_y=True)

fig.show()

In [34]:
#OV correlation test all
OV_corr_rains = df_OV_weather["rain_amount"].corr(df_OV_weather["Aantal_check_ins"])
OV_corr_wind = df_OV_weather["windspeed"].corr(df_OV_weather["Aantal_check_ins"])
OV_corr_temps = df_OV_weather["temperature"].corr(df_OV_weather["Aantal_check_ins"])

OV_corr_data = {"WeatherTypes": ["Rain", "Wind", "Temps"], "Correlation": [OV_corr_rains,OV_corr_wind,OV_corr_temps]}
df_OV_corr = pd.DataFrame(OV_corr_data)

fig = px.bar(df_OV_corr, x="WeatherTypes", y="Correlation", color="WeatherTypes",text_auto=True, title="Correlation between Weather condiditions and OV-checkins(All data)")
fig.show()

Investigate further in weekend data

In [39]:
df_OV_sum["dates"] = pd.to_datetime(df_OV_sum["dates"], format="%d-%m-%Y")

df_OV_weekends = df_OV_sum[(df_OV_sum["dates"].dt.dayofweek == 5) | (df_OV_sum["dates"].dt.dayofweek == 6)]
fig = px.line(df_OV_weekends, x="dates", y="Aantal_check_ins", title='Number of check-ins on weekend days 2023')
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Number of OV check-ins")

fig.show()

In [40]:
df_OV_weekends.rename(columns={"Datum":"dates"}, inplace=True)
df_OV_weather_weekends = df_OV_weekends.merge(df_weather_mean, on = "dates")

#OV correlation test weekends
OV_corr_rains_weekends = df_OV_weather_weekends["rain_amount"].corr(df_OV_weather_weekends["Aantal_check_ins"])
OV_corr_wind_weekends = df_OV_weather_weekends["windspeed"].corr(df_OV_weather_weekends["Aantal_check_ins"])
OV_corr_temps_weekends = df_OV_weather_weekends["temperature"].corr(df_OV_weather_weekends["Aantal_check_ins"])

OV_corr_data_weekends = {"WeatherTypes": ["Rain", "Wind", "Temps"], "Correlation": [OV_corr_rains_weekends,OV_corr_wind_weekends,OV_corr_temps_weekends]}
df_OV_corr_weekends = pd.DataFrame(OV_corr_data_weekends)

fig = px.bar(df_OV_corr_weekends, x="WeatherTypes", y="Correlation", color="WeatherTypes",text_auto=True, title="Correlation between Weather condiditions and OV-checkins(All data)")
fig.show()

/var/folders/j9/9tvcq7qs4gvf1y4c24tlh03h0000gn/T/ipykernel_60262/2017809589.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy










## Unused data right now (Seperate Sat and Sun)

In [73]:
df_OV_saturday = df_OV_sum[df_OV_sum["dates"].dt.dayofweek == 5]
df_OV_saturday["DayOfWeek"] = "Zaterdag"
df_OV_sunday = df_OV_sum[df_OV_sum["dates"].dt.dayofweek == 6]
df_OV_sunday["DayOfWeek"] = "Zondag"

combined_data = pd.concat([df_OV_saturday, df_OV_sunday])
combined_data.head()

fig = px.line(combined_data, x="dates", y="Aantal_check_ins", color="DayOfWeek", title='Number of check-ins on the weekend 2023')
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Number of OV check-ins")

fig.show()

/var/folders/j9/9tvcq7qs4gvf1y4c24tlh03h0000gn/T/ipykernel_11016/1278133868.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/j9/9tvcq7qs4gvf1y4c24tlh03h0000gn/T/ipykernel_11016/1278133868.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_OV_weather_saturday = df_OV_weather[(df_OV_weather["dates"].dt.dayofweek == 5)]
df_OV_weather_saturday.head()

In [ ]:
df_weekend_data = df_cong_grouped[(df_cong_grouped['DayOfWeek'] == 'Saturday') | (df_cong_grouped['DayOfWeek'] == 'Sunday')]
fig = px.line(df_weekend_data, x="dates", y="FileZwaarte", color="DayOfWeek")
fig.update_layout(title='File zwaarte per dag (Zaterdagen en Zondagen)', xaxis_title="Date", yaxis_title="Congestion severity (km*min)")

fig.show()